In [116]:
import pandas as pd
# import numpy as np
import requests as req
import sys
import time
# pip install holidays if you don't have the package already
import holidays
from datetime import datetime
from datetime import timedelta
# change the following filepath, file, and key name to your own
sys.path.insert(0, '../../API_Keys/')
from vantage_api import alpha_vantage_key

In [2]:
#below, be sure to change the name of 'apikey' to your key's name
def pull_all_stocks(df):
    stock_dict = {}
    for ticker in list(df['ticker'].unique()):
        json = req.get(url = "https://www.alphavantage.co/query",
                  params = {
                      'function': 'TIME_SERIES_DAILY',
                      'outputsize': 'full',
                      'symbol': ticker,
                      'apikey':alpha_vantage_key #change the name of the key to your key name
                  }).json()
        ticker_df = pd.DataFrame(json['Time Series (Daily)']).T
        ticker_df.rename(columns = {'1. open': 'open',
                                 '2. high': 'high',
                                 '3. low': 'low',
                                 '4. close': 'close',
                                 '5. volume': 'volume'}, inplace = True)
        if ticker_df.shape[0] > 2500:
            ticker_df = ticker_df.iloc[range(2500)] #Data beyond 2012 is unnecessary, this helps reduce file size
        stock_dict[f'{ticker}'] = ticker_df
        time.sleep(2) #Might need to be changed for long list of tickers
    return stock_dict


In [5]:
reps = pd.read_csv('data/cleaned_complete_congress_data.csv')

In [9]:
reps.head()

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
1,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_partial,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
2,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_full,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
3,2022-05-31,X,United States Steel Corporation,purchase,250000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
4,2022-05-20,PYPL,"PayPal Holdings, Inc. -",purchase,100000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL


In [130]:
mcconnell = reps[reps['last_name'] == 'McConnell']
mcconnell.head()

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
111,2022-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-03-14,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
355,2021-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-12-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
455,2021-09-03,WFC,WFC - Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-09-15,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
669,2021-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-06-07,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY


In [110]:
mcconnell.shape

(23, 14)

In [ ]:
#stocks = pd.DataFrame(columns = ['ticker','daily_time_series'])
#time_series_df.set_index('ticker', inplace = True)

In [145]:
stocks = pull_all_stocks(mcconnell)

In [146]:
stocks

{'WFC':                open     high      low    close    volume
 2022-07-22  43.1700  43.5300  42.6900  43.1700  15965371
 2022-07-21  42.4800  43.3700  42.2800  43.2800  17808953
 2022-07-20  42.5400  43.0250  42.3550  42.7200  19111193
 2022-07-19  41.8800  43.0700  41.7450  42.9000  25724214
 2022-07-18  41.9300  42.7900  41.0200  41.1900  17991317
 ...             ...      ...      ...      ...       ...
 2012-08-21  34.2400  34.7600  34.1000  34.3800  25280400
 2012-08-20  34.0300  34.2000  33.9800  34.0700  18147600
 2012-08-17  34.2500  34.3000  33.9600  34.0300  16927000
 2012-08-16  34.1200  34.2000  33.7800  34.1300  19596400
 2012-08-15  34.0100  34.1800  33.8300  33.9700  13766800
 
 [2500 rows x 5 columns],
 'VMC':                 open      high       low     close  volume
 2022-07-22  157.1600  157.8200  154.1300  155.6500  480375
 2022-07-21  153.8600  156.6500  153.1450  156.4600  592335
 2022-07-20  151.5200  154.7800  151.5200  154.5800  632510
 2022-07-19  150.6100 

In [147]:
stocks['IR'].loc['2022-07-22'].astype(float)

open           45.00
high           45.19
low            44.18
close          44.54
volume    2200156.00
Name: 2022-07-22, dtype: float64

In [57]:
mcconnell[mcconnell['transaction_date'] == '2019-09-01']

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents,price
2388,2019-09-01,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2019-09-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,NaN


In [148]:
stocks['WFC'].loc['2012-08-21']

open       34.2400
high       34.7600
low        34.1000
close      34.3800
volume    25280400
Name: 2012-08-21, dtype: object

In [111]:
mcconnell.dtypes

transaction_date     object
ticker               object
asset_description    object
type                 object
amount                int64
name                 object
disclosure_date      object
chamber              object
first_name           object
last_name            object
party                object
birthday             object
gender               object
represents           object
dtype: object

# Code to Pull Stock Prices
[Stackoverflow](https://stackoverflow.com/a/61426301) on mean in multipe columns. helped to get this written.[Stackoverflow](https://stackoverflow.com/a/61426301) on mean in multipe columns. helped to get this written.

## Transaction Date = Day Market Was Closed
Congresspeople sometimes entered transaction dates for which the market was closed. We presumed that means they entered the trade with their broker (or otherwise) on that date, and that it would be executed the next trading day. That's accounted for within a couple of try/except pairs.

[This stackoverflow](https://stackoverflow.com/a/61964191) helped with setting up the try/except. [This medium article](https://medium.com/analytics-vidhya/python-how-to-automate-bank-holidays-767de0d42d44) showed how us the holidays module. [This article](https://www.geeksforgeeks.org/python-holidays-library/) helped understand how to use it. [This](https://pynative.com/python-get-the-day-of-week/) showed how to get day of week to check if saturday or sunday so we can adjust day of transaction accordingly.

## Market Closed non-Holidays
2018-12-05 the market was closed for national day of mourning to mark [the passing of George Bush](https://heavy.com/news/2018/12/stock-market-closed-national-day-of-mourning/). Need to add it to the list of holidays.# Market Closed non-Holidays
2018-12-05 the market was closed for national day of mourning to mark [the passing of George Bush](https://heavy.com/news/2018/12/stock-market-closed-national-day-of-mourning/). Need to add it to the list of holidays.

We presume the holiday transaction date means that the orders were executed the next market day.

## Trades When Stocks Weren't Trading AKA Mitch McConnell Sells Something Not On the Exchange?
Some trades are registered as happening when stocks weren't trading. Specifically Ingersoll-Rand (IR) partial_sales by Mitch McConnell 2017-02-02 and 2016-06-09. 

The data on IR doesn't start until May 8, 2017. [This Motley Fool article](https://www.fool.com/investing/general/2016/01/04/is-ingersoll-rand-a-buy-for-2016.aspxhttps://www.fool.com/investing/general/2016/01/04/is-ingersoll-rand-a-buy-for-2016.aspx) makes it appear that Ingersoll-Rand was trading under the ticker TT (or perhaps that the part that wasn't spun into IR is trading as TT now).

Gardner Denver went public in May 2017. In May 2020, it combined with Ingersoll-Rand's HVAC business -- together they started trading under IR. See [this article](https://seekingalpha.com/article/4350361-new-ingersoll-rand-debuts-under-challenging-circumstanceshttps://seekingalpha.com/article/4350361-new-ingersoll-rand-debuts-under-challenging-circumstances).

This may be a noteworthy trade. The disclosure wasn't made until May 15, 2017 -- about the time of the appearance of IR on the stock exchange, and the time of Gardner Denver going public, but before they merged (at least, that it was announced).

Weird.

[This site](https://docs.python.org/2/library/datetime.html#datetime.timedeltahttps://docs.python.org/2/library/datetime.html#datetime.timedelta) showed us timedelta



In [194]:
us_holidays = holidays.UnitedStates()
error_dates = []
holiday_dates = []
weird_holidays = [] #holidays for which market wasn't open the next day
saturdays = []
sundays = []
still_unknown = []
mcconnell['trans_trading_day'] = 1 #assume the day is a trading day unless --> see below
mcconnell['trading_day'] = '' #establish the next date of trading, either equal to trans_date or after
for i in mcconnell.index:
    try:
        mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']].drop('volume').astype(float).mean()
        mcconnell.loc[i, 'trading_day'] = mcconnell.loc[i, 'transaction_date']
    except:
        trans_date = mcconnell.loc[i, 'transaction_date']
        trans_date_datetime = pd.to_datetime(mcconnell.loc[i, 'transaction_date'])
        error_dates.append(trans_date)
        mcconnell.loc[i, 'trans_trading_day'] = 0 #to track which transaction dates weren't trading dates
        
        date_plus_1 = str(pd.to_datetime(mcconnell.loc[i, 'transaction_date']) + timedelta(days = 1))[:10]
        date_plus_2 = str(pd.to_datetime(mcconnell.loc[i, 'transaction_date']) + timedelta(days = 2))[:10]
        date_plus_3 = str(pd.to_datetime(mcconnell.loc[i, 'transaction_date']) + timedelta(days = 3))[:10]

        if trans_date in us_holidays:
            holiday_dates.append(trans_date)
            try:
                #trying the following day.
                mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_1].drop('volume').astype(float).mean()
                mcconnell.loc[i, 'trading_day'] = date_plus_1
            except:
                weird_holidays.append(trans_date) #holidays with more than 1 day until the next trading day
        elif trans_date in ['2018-12-05']: #market closed these days for non-regular holidays (2018-12-05 nat'l day of morning for George Bush)
            try:
                #trying the following day.
                mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_1].drop('volume').astype(float).mean()
                mcconnell.loc[i, 'trading_day'] = date_plus_1
            except:
                weird_holidays.append(trans_date) #holidays with more than 1 day until the next trading day
        else:
            #check day of week, if saturday add 2 days, if sunday add 1 day
            if datetime.weekday(trans_date_datetime) == 5:
                saturdays.append(trans_date)
                if date_plus_2 not in us_holidays:
                    mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_2].drop('volume').astype(float).mean()
                    mcconnell.loc[i, 'trading_day'] = date_plus_2
                else:
                    mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_3].drop('volume').astype(float).mean()
                    mcconnell.loc[i, 'trading_day'] = date_plus_3
            # was here turing transaction to datetime in line before switching strategy
            elif datetime.weekday(trans_date_datetime) == 6:
                sundays.append(trans_date)
                if date_plus_1 not in us_holidays:
                    print(f'Transaction date: {trans_date}')
                    print(f'Date plus 1: {date_plus_1}')
                    mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_1].drop('volume').astype(float).mean()
                    mcconnell.loc[i, 'trading_day'] = date_plus_1
                else:
                    mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_2].drop('volume').astype(float).mean()
                    mcconnell.loc[i, 'trading_day'] = date_plus_2
            else:
                try: 
                    mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_1].drop('volume').astype(float).mean()
                    mcconnell.loc[i, 'trading_day'] = date_plus_1
                except:
                    still_unknown.append([mcconnell.loc[i, 'ticker'], trans_date])
                    print(f'{mcconnell.loc[i, "ticker"]} does not appear to trade on {trans_date}, which should be a day the exchange was open')
                    pass
    pass

IR does not appear to trade on 2017-02-02, which should be a day the exchange was open
IR does not appear to trade on 2016-06-09, which should be a day the exchange was open


/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_17438/4170938810.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcconnell['trans_trading_day'] = 1 #assume the day is a trading day unless --> see below
/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_17438/4170938810.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcconnell['trading_day'] = '' #establish the next date of trading, either equal to trans_date or after


In [185]:
saturdays

['2019-06-01']

In [186]:
sundays

['2019-09-01']

In [187]:
error_dates

['2019-09-01', '2019-06-01', '2018-12-05', '2017-02-02', '2016-06-09']

In [188]:
holiday_dates

[]

In [189]:
weird_holidays

[]

In [190]:
error_dates

['2019-09-01', '2019-06-01', '2018-12-05', '2017-02-02', '2016-06-09']

In [195]:
still_unknown

[['IR', '2017-02-02'], ['IR', '2016-06-09']]

In [191]:
mcconnell

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents,trading_day,price,trans_trading_day
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,2022-06-03,44.897475,1
111,2022-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-03-14,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,2022-03-03,51.772500,1
355,2021-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-12-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,2021-12-03,48.410000,1
455,2021-09-03,WFC,WFC - Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-09-15,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,2021-09-03,44.245000,1
669,2021-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-06-07,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,2021-06-03,46.625000,1
709,2021-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-03-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,2021-03-03,37.762500,1
828,2020-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-12-07,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,2020-12-03,29.095000,1
966,2020-09-02,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-09-11,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,2020-09-02,24.292500,1
1996,2020-03-04,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-03-11,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,2020-03-04,40.950000,1
2196,2019-12-04,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2019-12-12,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,2019-12-04,52.965000,1


In [192]:
stocks['IR']#.loc['2017-02-02']#.loc[mcconnell.loc[0, 'transaction_date']]

,open,high,low,close,volume
2022-07-22,45.0000,45.1900,44.1800,44.5400,2200156
2022-07-21,44.0700,44.9900,43.7300,44.9600,1633376
2022-07-20,43.2900,44.2400,43.1400,44.0700,2715744
2022-07-19,41.8300,43.8100,41.8300,43.2700,4499483
2022-07-18,41.5000,42.1600,40.9200,41.2200,3052687
...,...,...,...,...,...
2017-05-18,22.1900,23.6300,21.9800,22.5000,4075173
2017-05-17,22.1500,22.5100,21.6700,22.1000,1583699
2017-05-16,22.1900,22.4100,22.0500,22.1000,1289339
2017-05-15,21.0100,22.3000,21.0100,22.0200,3487278


In [180]:
stocks[mcconnell.loc[0, 'ticker']].loc['2017-02-02']#.loc[mcconnell.loc[0, 'transaction_date']]

open       55.5650
high       55.9600
low        55.4000
close      55.7500
volume    13044449
Name: 2017-02-02, dtype: object

In [ ]:
stocks[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, date_plus_2]]

In [174]:
stocks[mcconnell.loc[0, 'ticker']].loc[mcconnell.loc[0, 'transaction_date']].drop('volume').astype(float)

open     44.8300
high     45.3299
low      44.6100
close    44.8200
Name: 2022-06-03, dtype: float64

This pulls the average of open, high, low, and close. Using this because we have no idea when in the day the purchase was made.

In [176]:
stocks[mcconnell.loc[0, 'ticker']].loc[mcconnell.loc[0, 'transaction_date']].drop('volume').astype(float).mean()

44.897475